In [1]:
import pandas as pd
import random
import numpy as np

from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
df = pd.read_csv('../../dados/credit_data.csv')

In [45]:
print(df.shape)
df.head()

(2000, 6)


,i#clientid,income,age,loan,c#default,grupo
0,1,66155.925095,59.017015,8106.532131,0,0
1,2,34415.153966,48.117153,6564.745018,0,0
2,3,57317.170063,63.108049,8020.953296,0,0
3,4,42709.534201,45.751972,6103.642260,0,0
4,5,66952.688845,18.584336,8770.099235,1,0


### Amostragem aleatória simples

In [4]:
def amostragem_aleatoria_simples(df, n, seed):
    SEED = seed
    df_amostragem = df.sample(n=n, random_state=SEED)
    return df_amostragem

In [6]:
df_amostragem_aleatoria_simples = amostragem_aleatoria_simples(df, 1000, 1)
print(df_amostragem_aleatoria_simples.shape)
df_amostragem_aleatoria_simples.head()

(1000, 5)


,i#clientid,income,age,loan,c#default
674,675,34158.633968,29.421142,2911.408067,0
1699,1700,25789.742025,45.316211,4442.331780,0
1282,1283,59589.064289,20.609764,4191.715856,0
1315,1316,49908.291867,29.550940,2903.036128,0
1210,1211,69132.462579,33.471182,7621.410219,0


### Amostragem sistemática

In [7]:
def amostragem_sistematica(df, n):
    t = len(df)
    random.seed(1)
    amostragem = t // n
    valor_escolhido = random.randint(0, amostragem)
    numeros_escolhidos = np.arange(valor_escolhido, t, step=amostragem)
    amostra_sistematica = df.iloc[numeros_escolhidos]
    return amostra_sistematica

In [8]:
df_amostragem_sistematica = amostragem_sistematica(df, 1000)
print(df_amostragem_sistematica.shape)
df_amostragem_sistematica.head()

(1000, 5)


,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
2,3,57317.170063,63.108049,8020.953296,0
4,5,66952.688845,18.584336,8770.099235,1
6,7,48430.359613,26.809132,5722.581981,0
8,9,40654.892537,55.496853,4755.825280,0


### Amostragem por grupo

In [56]:
def amostragem_agrupamento(df, numero_grupos):
    total_por_grupo = len(df) / numero_grupos
    grupos = []
    id_grupo = 0
    contagem = 0
    for _ in df.iterrows():
        grupos.append(id_grupo)
        contagem += 1
        if contagem > total_por_grupo:
            contagem = 0
            id_grupo +=1
    df['grupo'] = grupos
    random.seed(8)
    numero_aleatorio = random.randint(0,9)
    df_agrupamento = df[df['grupo'] == numero_aleatorio]
    return df_agrupamento

In [59]:
df_amostragem_agrupamento = amostragem_agrupamento(df, 4)
print(df_amostragem_agrupamento.shape)

(497, 6)


### Amostragem estratificada

In [60]:
def amostragem_estratificada(df, percentual):
    split = StratifiedShuffleSplit(test_size=percentual)
    for _, y in split.split(df, df['c#default']):
        df_y = df.iloc[y]
    return df_y

In [61]:
df_amostragem_estratificada = amostragem_estratificada(df, 0.5)
print(df_amostragem_estratificada.shape)
df_amostragem_estratificada.head()

(1000, 6)


,i#clientid,income,age,loan,c#default,grupo
494,495,52534.785476,47.183619,3810.131842,0,0
100,101,51461.053167,36.651559,7292.264177,0,0
213,214,62658.221629,25.463160,3343.367161,0,0
596,597,61464.820644,36.408266,5099.087460,0,1
607,608,45652.059272,53.674341,5408.212129,0,1


### Amostragem de reservatório

In [62]:
def amostragem_reservatorio(df, amostras):
    stream = []
    for i in range(len(df)):
        stream.append(i)
        
    i = 0
    tamanho = len(df)
    
    reservatorio = [0] * amostras
    for i in range(amostras):
        reservatorio[i] = stream[i]
        
    while i < tamanho:
        j = random.randrange(i + 1)
        if j < amostras:
            reservatorio[j] = stream[i]
        i += 1
    
    return df.iloc[reservatorio]

In [63]:
df_amostragem_reservatorio = amostragem_reservatorio(df, 1000)
print(df_amostragem_reservatorio.shape)

(1000, 6)


In [64]:
df_amostragem_reservatorio.head()

,i#clientid,income,age,loan,c#default,grupo
1507,1508,32291.544548,39.174614,277.387568,0,3
1,2,34415.153966,48.117153,6564.745018,0,0
1831,1832,29750.294802,45.544455,3627.987077,0,3
3,4,42709.534201,45.751972,6103.642260,0,0
4,5,66952.688845,18.584336,8770.099235,1,0


### Comparativo entre as amostragens

In [66]:
#DF
print(df['age'].mean())
print(df['income'].mean())
print(df['loan'].mean())

40.80755937840458
45331.60001779333
4444.369694688262


In [67]:
#AleatóriaSimples
print(df_amostragem_aleatoria_simples['age'].mean())
print(df_amostragem_aleatoria_simples['income'].mean())
print(df_amostragem_aleatoria_simples['loan'].mean())

40.49552561124429
45563.26865376901
4449.4469004423645


In [68]:
#Sistemática
print(df_amostragem_sistematica['age'].mean())
print(df_amostragem_sistematica['income'].mean())
print(df_amostragem_sistematica['loan'].mean())

40.91117381141754
45691.49875066951
4506.787976426329


In [69]:
#PorGrupo
print(df_amostragem_agrupamento['age'].mean())
print(df_amostragem_agrupamento['income'].mean())
print(df_amostragem_agrupamento['loan'].mean())

40.10642021105454
46066.033301384974
4640.065674553214


In [70]:
#Reservatório
print(df_amostragem_reservatorio['age'].mean())
print(df_amostragem_reservatorio['income'].mean())
print(df_amostragem_reservatorio['loan'].mean())

40.62728637388653
45681.43481757729
4368.521166037777
